In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
from scipy.stats import norm
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import StandardScaler
from scipy import stats

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train_ID = train['Id']
test_ID = test['Id']
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

# df is the merged dataset (train + test)
df = pd.concat((train, test)).reset_index(drop=True)

C:\Users\mcui1\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## 2. Overall EDA (Distribution by variable)

In [2]:
# helper function to visualize distribution
def visdist(df,col):
    plt.subplot(1, 2, 1)
    plt.hist(df[col])
    plt.subplot(1, 2, 2)
    plt.scatter(df[col],df['SalePrice'])
    plt.tight_layout()
    plt.show()
    
cat_colnames = df.select_dtypes(include=['object']).columns
#print(cat_colnames, len(cat_colnames))
num_colnames = df.select_dtypes(exclude = ["object"]).columns
#print(num_colnames,len(num_colnames))

## 3. Data Cleaning: Missing Value Imputation

In [3]:
def clean(df):
    # **** what to do with Year and Month?
    for var in ['MSSubClass']:
        # later on, change to the actual string values
        df[var] = df[var].apply(str)
        
    # no garage, no bathrooms, etc., based on data description?  
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars','MasVnrArea','BsmtFinSF1','BsmtFinSF2'
           ,'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BsmtUnfSF','TotalBsmtSF'):
        df[col] = df[col].fillna(0).astype(int)
        
    # Replacing missing data with None, based on data description
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','BsmtQual',
            'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',"PoolQC"
           ,'Alley','Fence','MiscFeature','FireplaceQu','MasVnrType','Utilities']:
        df[col] = df[col].fillna('None')
            
    # Home functionality (Assume typical unless deductions are warranted): 
    # data description says NA means typical
    df['Functional'] = df['Functional'].fillna('Typ')
    
    # lot frontage: correlated with lot area (0.48) and neighborhoods (domain expertise)
    df = cleanLotFrontage(df)

    # ****** MSZoning, grouped by MSSubClass but I'm not exactly sure this is valid
    # did chi square test, wasn't too helpful, so many other categorical variables associated as much
    df['MSZoning'] = df.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
    
    common_vars = ['Exterior1st','Exterior2nd','SaleType','Electrical','KitchenQual']
    for var in common_vars:
        df[var] = df[var].fillna(df[var].mode()[0])
    
    return df

def cleanLotFrontage(df):
    df['LotArea_bin'] = pd.cut(df['LotArea'],50).apply(lambda x: x.mid)
    df['Lotfrontage_grouped'] = df.groupby(['Neighborhood','LotArea_bin'])['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    df['Lotfrontage_grouped'] = df.groupby(['Neighborhood'])['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    #df['LotFrontage_neigh'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    #df['LotFrontage_lotarea'] = df.groupby('LotArea')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    df['LotFrontage'] = df['Lotfrontage_grouped']
    df = df.drop(columns=['LotArea_bin','Lotfrontage_grouped']) 
    return df

In [4]:
def computeMissingness(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    # df.boxplot(column='LotFrontage',by='Neighborhood')
    # plt.scatter(x=df.LotArea,y=df.LotFrontage)
    return missing_data

In [5]:
# clean the data and compute the % of missingness to confirm
df = clean(df)
computeMissingness(df).head(10)

,Total,Percent
SalePrice,1459,0.499829
Electrical,0,0.000000
ExterCond,0,0.000000
ExterQual,0,0.000000
Exterior1st,0,0.000000
Exterior2nd,0,0.000000
Fence,0,0.000000
FireplaceQu,0,0.000000
Fireplaces,0,0.000000
Foundation,0,0.000000


In [6]:
#  Adding total sqfootage feature 
df['TotalSF']=df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

#  Adding total bathrooms feature
df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) +
                               df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))

#  Adding total porch sqfootage feature
df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] +
                              df['EnclosedPorch'] + df['ScreenPorch'] +
                              df['WoodDeckSF'])

In [7]:
categorical_features = df.select_dtypes(include=['object']).columns
print(categorical_features)
numerical_features = df.select_dtypes(exclude = ["object"]).columns
print(numerical_features)

print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
feat_num = df[numerical_features]
feat_cat = df[categorical_features]

Index(['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC',
       'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig',
       'LotShape', 'MSSubClass', 'MSZoning', 'MasVnrType', 'MiscFeature',
       'Neighborhood', 'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'Street', 'Utilities'],
      dtype='object')
Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GarageYrBlt', 'GrLivArea', 'HalfBath', 'KitchenAbvGr', 'LotArea',
       'LotFrontage', 'LowQua

In [8]:
# we need to transform those numerical features where skewness is > 0.5
# abs(skewness)>1  highly sknewed 
# 1 > abs(skewness) > 0.5 moderately sknewed
# we are taking the conservative approach and adjusting for moderately skewed 

skewness = feat_num.apply(lambda x: skew(x))
skewness = skewness[abs(skewness) > 0.5]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
print("Mean skewnees: {}".format(np.mean(skewness)))

from scipy.stats import skew 
skewness.sort_values(ascending=False)

There are 29 skewed numerical features to Box Cox transform
Mean skewnees: 3.8905513337095416


MiscVal           21.947195
PoolArea          16.898328
LotArea           12.822431
LowQualFinSF      12.088761
3SsnPorch         11.376065
KitchenAbvGr       4.302254
BsmtFinSF2         4.146143
EnclosedPorch      4.003891
ScreenPorch        3.946694
BsmtHalfBath       3.931594
MasVnrArea         2.613592
OpenPorchSF        2.535114
WoodDeckSF         1.842433
TotalSF            1.511479
LotFrontage        1.505704
1stFlrSF           1.469604
BsmtFinSF1         1.425230
Total_porch_sf     1.376649
GrLivArea          1.269358
TotalBsmtSF        1.156894
BsmtUnfSF          0.919339
2ndFlrSF           0.861675
TotRmsAbvGrd       0.758367
Fireplaces         0.733495
HalfBath           0.694566
BsmtFullBath       0.624832
OverallCond        0.570312
YearBuilt         -0.599806
GarageYrBlt       -3.906205
dtype: float64

In [9]:
from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    feat_num[feat] = boxcox1p(feat_num[feat], boxcox_normmax(feat_num[feat] + 1))
    df[feat] = boxcox1p(df[feat], boxcox_normmax(df[feat] + 1))
    
skewness = feat_num.apply(lambda x: skew(x))
skewness = skewness[abs(skewness) > 0.5]

print("There are {} skewed numerical features after Box Cox transform".format(skewness.shape[0]))
print("Mean skewnees: {}".format(np.mean(skewness)))
skewness.sort_values(ascending=False)

C:\Users\mcui1\AppData\Roaming\Python\Python36\site-packages\scipy\stats\stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
C:\Users\mcui1\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\mcui1\AppData\Roaming\Python\Python36\site-packages\scipy\stats\stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


There are 17 skewed numerical features after Box Cox transform
Mean skewnees: 3.465448337560018


PoolArea         14.985994
3SsnPorch         8.865144
LowQualFinSF      8.495602
MiscVal           5.239894
BsmtHalfBath      3.780960
KitchenAbvGr      3.779896
ScreenPorch       3.153911
BsmtFinSF2        2.579468
EnclosedPorch     2.150157
MasVnrArea        0.976354
2ndFlrSF          0.894536
WoodDeckSF        0.784768
HalfBath          0.730771
GarageYrBlt       0.701201
OpenPorchSF       0.621025
BsmtFullBath      0.618419
Fireplaces        0.554522
dtype: float64

### Dummify 

In [10]:
df = df.drop("SalePrice", axis = 1)
final_features = pd.get_dummies(df)

In [11]:
# Apply transformation
train.SalePrice = np.log1p(train.SalePrice)

# New prediction
y_train = train.SalePrice.values
y_train_orig = train.SalePrice

In [12]:
print(final_features.shape)
X = final_features.iloc[:len(y_train), :]
X_test = final_features.iloc[len(y_train):, :]
X.shape, y_train.shape, X_test.shape


print(X.shape,y_train.shape,X_test.shape)

(2919, 321)
(1460, 321) (1460,) (1459, 321)


### Re-run missing data to see if there is any missing data 

In [13]:
# computemissingdata(df)
# # so we're all set!

### Modeling 

In [14]:
from datetime import datetime
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error , make_scorer
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [15]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

# model scoring and validation function
def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y,scoring="neg_mean_squared_error",cv=kfolds))
    return (rmse)

# rmsle scoring function
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [18]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4, #was 3
                                       learning_rate=0.01, 
                                       n_estimators=8000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2, # 'was 0.2'
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )

In [19]:
# This is a range of values that the model considers each time in runs a CV
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]

In [20]:
# Kernel Ridge Regression : made robust to outliers
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))

# LASSO Regression : made robust to outliers
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, 
                    alphas=alphas2,random_state=42, cv=kfolds))

# Elastic Net Regression : made robust to outliers
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, 
                         alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))


stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, lightgbm),
                                meta_regressor=elasticnet,
                                use_features_in_secondary=True)

# store models, scores and prediction values 
models = {'Ridge': ridge,
          'Lasso': lasso, 
          'ElasticNet': elasticnet,
          'lightgbm': lightgbm}
#           'xgboost': xgboost}
predictions = {}
scores = {}

### Train model 

In [22]:
for name, model in models.items():
    y = y_train
    model.fit(X, y)
    predictions[name] = np.expm1(model.predict(X))
    
    score = cv_rmse(model, X=X)
    scores[name] = (score.mean(), score.std())

In [23]:
# get the performance of each model on training data(validation set)
print('---- Score with CV_RMSLE-----')
score = cv_rmse(ridge)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(elasticnet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

score = cv_rmse(lightgbm)
print("lightgbm score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

# score = cv_rmse(xgboost)
# print("xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


#Fit the training data X, y
print('----START Fit----',datetime.now())
print('Elasticnet')
elastic_model = elasticnet.fit(X, y)
print('Lasso')
lasso_model = lasso.fit(X, y)
print('Ridge')
ridge_model = ridge.fit(X, y)
print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)

# print('xgboost')
# xgb_model_full_data = xgboost.fit(X, y)


print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))

---- Score with CV_RMSLE-----
Ridge score: 0.1261 (0.0282)

Lasso score: 0.1237 (0.0283)

ElasticNet score: 0.1235 (0.0283)

lightgbm score: 0.1194 (0.0187)

----START Fit---- 2019-07-25 18:03:32.690611
Elasticnet
Lasso
Ridge
lightgbm
stack_gen


In [24]:
def blend_models_predict(X):
    return ((0.25  * elastic_model.predict(X)) + \
            (0.25 * lasso_model.predict(X)) + \
            (0.2 * ridge_model.predict(X)) + \
            (0.10 * lgb_model_full_data.predict(X)) + \
#             (0.1 * xgb_model_full_data.predict(X)) + \
            (0.2 * stack_gen_model.predict(np.array(X))))

In [25]:
print('RMSLE score on train data:')
print(rmsle(y, blend_models_predict(X)))

RMSLE score on train data:
0.1018122754274746


In [28]:
print('Predict submission')
submission = pd.read_csv("./sample_submission.csv")
submission.iloc[:,1] = (np.expm1(blend_models_predict(X_test)))

Predict submission


In [29]:
submission.to_csv("submission.csv", index=False)